In [29]:
import pandas as pd
from pp_netlib.network import Network
datafile = "strep_MLST.tsv"
data = pd.read_csv(datafile, sep= "\t", header=0)

## cleaning up the data a little
data.drop("clonal_complex", axis = 1, inplace=True)
data["ST"] = "ST_" + data["ST"].astype(str)
data.index = data["ST"]
data.drop("ST", axis = 1, inplace=True)
data = data.groupby(data.index)["aroE","gdh","gki","recP","spi","xpt","ddl"].apply(list)

,aroE,gdh,gki,recP,spi,xpt,ddl
ST,,,,,,,
ST_1,1,1,1,1,1,1,1
ST_2,1,1,4,1,18,13,18
ST_3,1,5,1,8,14,11,14
ST_4,1,5,2,20,1,1,1
ST_5,1,5,2,11,9,3,51
...,...,...,...,...,...,...,...
ST_17933,5,130,4,2,10,1,6
ST_17934,8,11,10,1,6,8,31
ST_17935,10,12,2,1,152,28,181


In [12]:
mult = data.dot(data.T)
mult.shape

(17915, 17915)

In [30]:
mult

array([[     7,     56,     54, ...,    386,    823,    446],
       [    56,    836,    665, ...,   6389,   3987,   6490],
       [    54,    665,    604, ...,   5050,   3316,   4990],
       ...,
       [   386,   6389,   5050, ...,  56898,  32711,  51397],
       [   823,   3987,   3316, ...,  32711, 401229,  56435],
       [   446,   6490,   4990, ...,  51397,  56435, 111348]])